In [0]:
# This notebook is meant to extract the data from sklearn.datasets and ingest it into a table in the UC

In [0]:
from sklearn import datasets
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id

In [0]:
dbutils.widgets.text("catalog_name", "pedroz_e2edata_dev")
catalog_name = dbutils.widgets.get("catalog_name")

In [0]:
iris_data = datasets.load_iris(as_frame=True)
df_iris = pd.DataFrame(data = iris_data['data'], columns = iris_data['feature_names'])
df_iris.columns = df_iris.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_iris['species'] = iris_data['target']
df_iris.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [0]:
df_iris['id'] = range(1, len(df_iris) + 1)
df_iris.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species,id
0,5.1,3.5,1.4,0.2,0,1
1,4.9,3.0,1.4,0.2,0,2
2,4.7,3.2,1.3,0.2,0,3
3,4.6,3.1,1.5,0.2,0,4
4,5.0,3.6,1.4,0.2,0,5


In [0]:
spark_df_iris = spark.createDataFrame(df_iris)

In [0]:
try:
    display(spark.table(f"{catalog_name}.default.iris_data").limit(5))
    table_exists = True
except:
    table_exists = False

sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species,id
5.1,3.5,1.4,0.2,0,1
4.9,3.0,1.4,0.2,0,2
4.7,3.2,1.3,0.2,0,3
4.6,3.1,1.5,0.2,0,4
5.0,3.6,1.4,0.2,0,5


In [0]:
if not table_exists:
    spark_df_iris.write.mode("overwrite").saveAsTable(f"{catalog_name}.default.iris_data")
    spark.sql(f"ALTER TABLE {catalog_name}.default.iris_data ALTER COLUMN id SET NOT NULL")
    spark.sql(f"ALTER TABLE {catalog_name}.default.iris_data ADD CONSTRAINT pk_id PRIMARY KEY (id)")
    print("Created table and added a primary key to it")
else:
    spark_df_iris.write.mode("overwrite").saveAsTable(f"{catalog_name}.default.iris_data")
    print("Overwrote table data")

Overwrote table data


In [0]:
df = spark.sql(f"SELECT * FROM {catalog_name}.default.iris_data")
display(df.limit(5))

sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species,id
5.1,3.5,1.4,0.2,0,1
4.9,3.0,1.4,0.2,0,2
4.7,3.2,1.3,0.2,0,3
4.6,3.1,1.5,0.2,0,4
5.0,3.6,1.4,0.2,0,5
